In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Step 2: Define a dictionary to map class indices to disease names
class_labels = {
    0: 'Acne and Rosacea Photos',
    1: 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
    2: 'Atopic Dermatitis Photos',
    3: 'Bullous Disease Photos',
    4: 'Cellulitis Impetigo and other Bacterial Infections',
    5: 'Eczema Photos',
    6: 'Exanthems and Drug Eruptions',
    7: 'Hair Loss Photos Alopecia and other Hair Diseases',
    8: 'Herpes HPV and other STDs Photos',
    9: 'Light Diseases and Disorders of Pigmentation',
    10: 'Lupus and other Connective Tissue diseases',
    11: 'Melanoma Skin Cancer Nevi and Moles',
    12: 'Nail Fungus and other Nail Disease',
    13: 'Poison Ivy Photos and other Contact Dermatitis',
    14: 'Psoriasis pictures Lichen Planus and related diseases',
    15: 'Scabies Lyme Disease and other Infestations and Bites',
    16: 'Seborrheic Keratoses and other Benign Tumors',
    17: 'Systemic Disease',
    18: 'Tinea Ringworm Candidiasis and other Fungal Infections',
    19: 'Urticaria Hives',
    20: 'Vascular Tumors',
    21: 'Vasculitis Photos',
    22: 'Warts Molluscum and other Viral Infections'
}

In [3]:
IMAGE_SIZE = (299, 299)  # InceptionV3 input size
BATCH_SIZE = 128
NUM_CLASSES = len(class_labels) 

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [8]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
train_generator = train_datagen.flow_from_directory(
    'D:/skin dataset/train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 15550 images belonging to 23 classes.


In [9]:
test_generator = test_datagen.flow_from_directory(
    'D:/skin dataset/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 3931 images belonging to 23 classes.


In [11]:
# Create an InceptionV3 base model with pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


87910968/87910968 [==============================] - 87s 1us/step


In [12]:
# Train the model
epochs = 50
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    epochs=epochs
)

Epoch 1/50
121/121 [==============================] - 1532s 12s/step - loss: 2.8308 - accuracy: 0.1869 - val_loss: 2.5810 - val_accuracy: 0.2542
Epoch 2/50
121/121 [==============================] - 1249s 10s/step - loss: 2.5897 - accuracy: 0.2437 - val_loss: 2.4645 - val_accuracy: 0.2854
Epoch 3/50
121/121 [==============================] - 1201s 10s/step - loss: 2.4898 - accuracy: 0.2714 - val_loss: 2.4082 - val_accuracy: 0.3013
Epoch 4/50
121/121 [==============================] - 1156s 10s/step - loss: 2.4311 - accuracy: 0.2852 - val_loss: 2.3721 - val_accuracy: 0.3055
Epoch 5/50
121/121 [==============================] - 1195s 10s/step - loss: 2.3891 - accuracy: 0.2933 - val_loss: 2.3307 - val_accuracy: 0.3208
Epoch 6/50
121/121 [==============================] - 1207s 10s/step - loss: 2.3444 - accuracy: 0.3041 - val_loss: 2.3001 - val_accuracy: 0.3299
Epoch 7/50
121/121 [==============================] - 1222s 10s/step - loss: 2.3081 - accuracy: 0.3195 - val_loss: 2.2793 - val_ac

In [13]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Save the model
model.save('skin_disease_classifier_inceptionv3.h5')
model.save('skin_disease_classifier_inceptionv3.keras')

31/31 [==============================] - 220s 7s/step - loss: 1.9688 - accuracy: 0.4246
Test Loss: 1.9688286781311035
Test Accuracy: 0.4245738983154297


c:\Users\bhava\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
